In [2]:
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
from webdriver_manager.chrome import ChromeDriverManager
from scrapy import Selector
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [5]:
options = webdriver.ChromeOptions()

options.page_load_strategy = 'normal'
options.add_argument('--remote-debugging-pipe')
driver = webdriver.Chrome(options=options)
driver.get('https://finance.yahoo.com/quote/NVDA/history/')
# Interakcja ze stroną, w celu wykonania akcji JavaScript
time.sleep(2) 
driver.find_element(By.XPATH,'//*[@id="scroll-down-btn"]').click()
time.sleep(2)
driver.find_element(By.XPATH,'//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[1]').click()
time.sleep(2)
driver.find_element(By.XPATH,'//*[@id="nimbus-app"]/section/section/section/article/div[1]/div[1]/div[1]/button/span').click()
time.sleep(2)
driver.find_element(By.XPATH,'/html/body/div[1]/main/section/section/section/article/div[1]/div[1]/div[1]/div/div/div[2]/section/div[1]/button[7]').click()
time.sleep(2)
# Zapisanie źródła strony po wykonaniu działań JavaScript
html = driver.page_source

# Zamknięcie przeglądarki
driver.quit()

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())


In [7]:
# Wyszukiwanie tabel w kodzie HTML
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['table', 'svelte-ewueuo']


In [ ]:
# Weryfikacja danych w tabeli
table = soup.find('table',{'class' : 'svelte-ewueuo'})
print(table)

In [13]:
# Zapisanie nazw kolumn
header = []
rows = []
for i, row in enumerate(table.find_all('tr')):
    if i == 0:
        header = [el.text.strip() for el in row.find_all('th')]
    else:
        rows.append([el.text.strip() for el in row.find_all('td')])
print(header)

['Date', 'Open', 'High', 'Low', 'Close    Close price adjusted for splits.', 'Adj Close    Adjusted close price adjusted for splits and dividend and/or capital gain distributions.', 'Volume']


In [9]:
# Zapisywanie danych z tabeli w nowym DataFrame
data = []
for row in table.find_all('tr'):
    row_data = []
    for cell in row.find_all('td'):
        row_data.append(cell.text)
    data.append(row_data)

In [10]:
df = pd.DataFrame(data)
print(df)

                 0       1       2       3       4       5            6
0             None    None    None    None    None    None         None
1     Jul 16, 2024  128.41  129.04  124.58  126.10  126.10  145,289,215
2     Jul 15, 2024  130.56  131.39  127.18  128.44  128.44  207,743,700
3     Jul 12, 2024  128.26  131.92  127.22  129.24  129.24  252,103,100
4     Jul 11, 2024  135.75  136.15  127.05  127.40  127.40  374,782,700
...            ...     ...     ...     ...     ...     ...          ...
1277  Jul 22, 2019    4.23    4.30    4.22    4.28    4.26  315,940,000
1278  Jul 19, 2019    4.29    4.30    4.20    4.21    4.19  344,896,000
1279  Jul 18, 2019    4.25    4.32    4.20    4.25    4.23  442,192,000
1280  Jul 17, 2019    4.18    4.25    4.15    4.24    4.22  348,204,000
1281  Jul 16, 2019    4.17    4.20    4.12    4.18    4.15  341,444,000

[1282 rows x 7 columns]


In [14]:
# Wklejenie poprzednio zapisanych nazw kolumn
df.columns = [header]
df.head()

,Date,Open,High,Low,Close Close price adjusted for splits.,Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.,Volume
0,None,None,None,None,None,None,None
1,"Jul 16, 2024",128.41,129.04,124.58,126.10,126.10,"145,289,215"
2,"Jul 15, 2024",130.56,131.39,127.18,128.44,128.44,"207,743,700"
3,"Jul 12, 2024",128.26,131.92,127.22,129.24,129.24,"252,103,100"
4,"Jul 11, 2024",135.75,136.15,127.05,127.40,127.40,"374,782,700"


In [15]:
df.drop(df.columns[5], axis=1, inplace=True)

In [16]:
df.head()

,Date,Open,High,Low,Close Close price adjusted for splits.,Volume
0,None,None,None,None,None,None
1,"Jul 16, 2024",128.41,129.04,124.58,126.10,"145,289,215"
2,"Jul 15, 2024",130.56,131.39,127.18,128.44,"207,743,700"
3,"Jul 12, 2024",128.26,131.92,127.22,129.24,"252,103,100"
4,"Jul 11, 2024",135.75,136.15,127.05,127.40,"374,782,700"


In [19]:
df.rename(index=str, columns={"Close    Close price adjusted for splits.": "Close"}, inplace=True)

In [20]:
print(df.columns)

MultiIndex([(  'Date',),
            (  'Open',),
            (  'High',),
            (   'Low',),
            ( 'Close',),
            ('Volume',)],
           )


In [21]:
# Usunięcie krotek z nazw kolumn
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

In [22]:
df = df.iloc[1:]

In [23]:
df.head(10)

,Date,Open,High,Low,Close,Volume
1,"Jul 16, 2024",128.41,129.04,124.58,126.10,"145,289,215"
2,"Jul 15, 2024",130.56,131.39,127.18,128.44,"207,743,700"
3,"Jul 12, 2024",128.26,131.92,127.22,129.24,"252,103,100"
4,"Jul 11, 2024",135.75,136.15,127.05,127.40,"374,782,700"
5,"Jul 10, 2024",134.03,135.10,132.42,134.91,"248,978,600"
6,"Jul 9, 2024",130.35,133.82,128.65,131.38,"285,366,600"
7,"Jul 8, 2024",127.49,130.77,127.04,128.20,"237,677,300"
8,"Jul 5, 2024",127.38,128.85,125.68,125.83,"214,176,700"
9,"Jul 3, 2024",121.66,128.28,121.36,128.28,"215,749,000"
10,"Jul 2, 2024",121.13,123.41,121.03,122.67,"218,374,000"


In [24]:
# Zmiana formatu zapisu daty
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
df['Volume'] = df.Volume.str.replace(',','')

In [25]:
df.head(10)

,Date,Open,High,Low,Close,Volume
1,2024-07-16,128.41,129.04,124.58,126.10,145289215
2,2024-07-15,130.56,131.39,127.18,128.44,207743700
3,2024-07-12,128.26,131.92,127.22,129.24,252103100
4,2024-07-11,135.75,136.15,127.05,127.40,374782700
5,2024-07-10,134.03,135.10,132.42,134.91,248978600
6,2024-07-09,130.35,133.82,128.65,131.38,285366600
7,2024-07-08,127.49,130.77,127.04,128.20,237677300
8,2024-07-05,127.38,128.85,125.68,125.83,214176700
9,2024-07-03,121.66,128.28,121.36,128.28,215749000
10,2024-07-02,121.13,123.41,121.03,122.67,218374000


In [180]:
# Zapis danych do pliku csv
df.to_csv('NVIDIA_stock.csv', index = False)